In [1]:
import os
print(os.getcwd())

from ib_insync import *
import src.floor_ceiling_regime as fcr
from trend_viewer import *
util.startLoop()
port = 4001
ib = IB()
ib.connect('127.0.0.1', port, clientId=0)

c:\Users\Brian\_projects\backtest_notebook\notes


<IB connected to 127.0.0.1:4001 clientId=0>

In [2]:
%matplotlib inline

In [3]:
c = ib.reqContractDetails(Contract(symbol='MES', secType='FUT', includeExpired=False))
c_df = util.df(c)
c_df

,contract,marketName,minTick,orderTypes,validExchanges,priceMagnifier,underConId,longName,contractMonth,industry,...,callable,putable,coupon,convertible,maturity,issueDate,nextOptionDate,nextOptionType,nextOptionPartial,notes
0,"Contract(secType='FUT', conId=654503314, symbo...",MES,0.25,"ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,B...",CME,1,362673777,Micro E-Mini S&P 500 Stock Price Index,202412,,...,False,False,0,False,,,,,False,
1,"Contract(secType='FUT', conId=672387462, symbo...",MES,0.25,"ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,B...",CME,1,362673777,Micro E-Mini S&P 500 Stock Price Index,202503,,...,False,False,0,False,,,,,False,
2,"Contract(secType='FUT', conId=691171673, symbo...",MES,0.25,"ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,B...",CME,1,362673777,Micro E-Mini S&P 500 Stock Price Index,202506,,...,False,False,0,False,,,,,False,
3,"Contract(secType='FUT', conId=711280067, symbo...",MES,0.25,"ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,B...",CME,1,362673777,Micro E-Mini S&P 500 Stock Price Index,202509,,...,False,False,0,False,,,,,False,
4,"Contract(secType='FUT', conId=730283085, symbo...",MES,0.25,"ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,B...",CME,1,362673777,Micro E-Mini S&P 500 Stock Price Index,202512,,...,False,False,0,False,,,,,False,


In [4]:
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import strategy.strategies as strategy
import strategy.indicators as indicator

def run(
    symbol, 
    sec_type, 
    interval='1 min', 
    duration='1 D', 
    use_rth=True, 
    keep_up_to_date=True, 
    band_window=256, 
    plot_window=600, 
    peak_window=None
):
    band_window = band_window
    contracts = ib.reqContractDetails(Contract(localSymbol=symbol, secType=sec_type, includeExpired=False))[0]
    contract = contracts.contract
    history_kwargs = {
        "contract": contract,
        "endDateTime": '',
        "durationStr": duration,
        "barSizeSetting": interval,
        "whatToShow": 'TRADES',
        "useRTH": use_rth,
        "formatDate": 1,
    }
    first_run_bars = ib.reqHistoricalData(**history_kwargs)
    _bars = ib.reqHistoricalData(keepUpToDate=keep_up_to_date, **history_kwargs)
    # indicator.TradingRange(high_band_pct=.40, low_band_pct=.61, window=band_window),
    strat = strategy.Custom(
        indicators=[
            indicator.Regime(),
        ]
    )
    
    def onBarUpdate(bars, hasNewBar):
        title = f'{bars.contract.symbol} {bars[-2].date}'
        if hasNewBar:
            # exclude the last bar because it is not complete
            prices = util.df(bars)
            # Exclude latest bar if it is not complete
            prices = prices[:-1]
            prices.index = prices.date 
            prices = prices.drop(columns=['date'])
            strat.update(prices.close.copy())
            price_data, ax = plot_fc_tables(title, strat, peak_window, plot_window, band_window, bandFunc=addBand)
            plt.show()
            clear_output(wait=True)

        
    if keep_up_to_date:
        onBarUpdate(first_run_bars, True)
        
        _bars.updateEvent += onBarUpdate
        while True:
            ib.sleep(100000)
        ib.cancelHistoricalData(_bars)
    else:
        onBarUpdate(_bars, True)
    
# calculate RSI run through floor ceiling?



In [5]:
"""
In a strong trend, the maximum retracement for an equity/index is usually 23.6% or 38.2%. 
In a weaker trend, one can generally expect the stock to retrace no more than 61.8% or 76.4% of its value
"""

# 8, 13, 21, 34, 55, 89, 144, 233, 377, 610
# # TODO why is band_window not bound in the plot_fc_tables function?
# run(
#     symbol='SMCI', 
#     sec_type='STK',
#     interval='1 min', 
#     duration='5 D',
#     use_rth=True,
#     keep_up_to_date=False,
#     band_window=233,

#     plot_window=1000,
# )
#     # peak_window=5,

# TODO translate ib qt table to a watchlist for pullback strategy

try:
    ib.cancelHistoricalData(bars)
except:
    pass
# MCLX4 MGCZ4
bars = run_profile(
    deps=(ib, Contract, strategy, indicator, util, clear_output),
    bandFunc=addBandAggregatePeakConcat,
    symbol='MGCZ4', 
    sec_type='FUT',
    interval='15 mins', 
    duration='15 D',
    use_rth=False,
    keep_up_to_date=True,
    band_window=233,
    peak_window=3,
    plot_window=1000,
    figsize=(15, 10),
    fast_band=True
)


Peer closed connection.


ConnectionError: Socket disconnect

: 